In [5]:

using ITensors
using PastaQ
using ITensorGLMakie
# using ITensorUnicodePlots
using JuliaFormatter

In [ ]:
enable_format(latex_symbols=true)

## Creating a one-site state
We will now create some simple one-site states and operators using the ITensor library.

In [ ]:

# Define a 2-dimensional Hilbert space with index i
i = Index(2)

# Create the |Z+> state
Zp = ITensor([1, 0], i)
Zp


## Creating the |Z-> and |X±> states

In [ ]:

Zm = ITensor([0, 1], i)
Xp = ITensor([1, 1]/√2, i)
Xm = ITensor([1, -1]/√2, i)

# Display the states
(Zm, Xp, Xm)


## Inner product and state manipulations
We can now calculate inner products and manipulate states.

In [ ]:

# Calculate inner product between |Z+> and |X+>
inner_product = (dag(Zp) * Xp)[]
inner_product


## Visualization
Use UnicodePlots for simple visualization of the tensors.

In [ ]:

# Visualize the inner product
@visualize dag(Zp) * Xp


## Priming indices and using operators

In [ ]:

# Prime the index i and check the difference
i_prime = prime(i)
Z = ITensor(i_prime, i)
Z[i_prime=>1, i=>1] = 1
Z[i_prime=>2, i=>2] = -1
Z

## Applying operators
Let's apply operators to states and observe the results.

In [ ]:

# Applying X operator to |Z+>
i = Index(2, "S=1/2")
Zp = state("Zp",i)
Zm = state("Zm",i)
X = op("X", i)
Zp_applied = X * Zp
Zp_applied


In [ ]:
(dag(Zm)' * X * Zp)[]

In [ ]:
apply(X,Zp) == noprime(X * Zp)

## Multi-site states
We will now create and work with two-site states.

In [ ]:

# Create two-site indices
i1 = Index(2, "S=1/2,i1")
i2 = Index(2, "S=1/2,i2")

# Create a two-site state |Z+Z->
Zp1 = state("Zp", i1)
Zp2 = state("Zp", i2)
Zm2 = state("Zm", i2)
Zm1 = state("Zm",i1)
# Los hemos definido con indices distintos así que juntarlos es como un producto tensorial
ZpZm = Zp1 * Zm2
ZpZm2 = ITensor(i1,i2)
ZpZm2[i1=>1,i2=>2] = 1
ZpZm == ZpZm2


In [ ]:
ψ = ITensor(i1,i2)
ψ[i1=>1, i2=>2] = 1/sqrt(2)
ψ[i1=>2, i2=>1] = 1/sqrt(2)
ψ2 = (Zm1*Zp2 + Zm2*Zp1)/sqrt(2)

In [ ]:
ψ2 == ψ

## Performing SVD and Tensor Factorizations
We can use Singular Value Decomposition (SVD) to factorize tensors.

In [ ]:

# Perform SVD on the two-site state ZpZm
_, S, _ = svd(ZpZm, i1) #Why here we write index i1?
diag(S)  # Show the singular values


Let's try now the SVD of one of the bell states. S should be S = [1/√2,  1/√2]

In [18]:
ψ2 = (Zm1*Zp2 + Zp1*Zm2)/√2
_,S,_ = svd(ψ2, i1);

In [ ]:
display(matrix(S))

## Working with Hamiltonians
Now we will define a simple two-site Hamiltonian.

In [ ]:
Id1 = op("Id",i1)
Z1 = op("Z",i1)
X1 = op("X",i1)
Id2 = op("Id",i2)
Z2 = op("Z",i2)
X2 = op("X",i2)
ZZ = Z1*Z2
XI = X1*Id2
IX = Id1*X2

h = 0.5
H = -ZZ + h*(XI + IX)

In [ ]:
inner_product = inner(Zp1' * Zm2', H, Zp1 * Zm2)
inner_product

In [ ]:
inner(Zp1'*Zp2', H, Zp1*Zp2)

In [ ]:
D,_ = eigen(H)
real(diag(D))

## Custom two site operator 
We are going to define the RXX gate we will need sometime

In [ ]:

import ITensors:op
function op(
    ::OpName"RXX",
    ::SiteType"S=1/2";θ)
    c = cos(θ/2)
    s = sin(θ/2)
    return [
        c 0 0 -im*s
        0 c -im*s 0
        0 -im*s c 0
        -im*s 0 0 c]
end

In [ ]:
RXX = op("RXX", i1,i2,θ = pi/2)
@visualize RXX

## Defining density matrices


In [ ]:
ψ = (Zp1*Zp2+Zm1*Zm2)/sqrt(2)
@visualize ψ   

In [ ]:
ρ = ψ*dag(ψ')

In [ ]:
n = 5
i = [Index(2,"S=1/2")
    for j in 1:n];

Zp = MPS(i, "Zp");
Zm = MPS(i, "Zm");

maxlinkdim(Zp) == 1

In [ ]:
ψ2 = (Zp + Zm)/√2
maxlinkdim(ψ2)
@visualize ψ2

In [ ]:
inner(Zp,Zp)